In [7]:
import requests
from requests import Response
from pydantic import BaseModel,Field,RootModel,field_validator
import pandas as pd

youbike_url = "https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json"
res:Response = requests.get(youbike_url)

if res.status_code == 200:
    print("Download Success")
else:
    print("Download Failure") 


class Site(BaseModel):
    站名:str = Field(alias='sna')
    地區:str = Field(alias = 'sarea')
    時間:str = Field(alias = 'mday')
    地點:str = Field(alias = 'ar')
    總車輛數:int = Field(alias='total')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')    
    可借:int = Field(alias='available_rent_bikes')
    可還:int = Field(alias='available_return_bikes')
    狀態:bool = Field(alias='act')
    
    @field_validator("站名")
    @classmethod
    def removefilter(cls,value:str)->str:
        return value.split("_")[-1]
    
class YouBike(RootModel):
    root: list[Site]
    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]
    

data:YouBike = YouBike.model_validate_json(res.text)
all_sites:list[dict]= data.model_dump()

df1 = pd.DataFrame(all_sites)
df1

Download Success


,站名,地區,時間,地點,總車輛數,經度,緯度,可借,可還,狀態
0,捷運科技大樓站,大安區,2024-06-17 19:21:21,復興南路二段235號前,28,121.54360,25.02605,1,27,True
1,復興南路二段273號前,大安區,2024-06-17 19:21:21,復興南路二段273號西側,21,121.54357,25.02565,9,12,True
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,16,121.54124,25.02429,2,14,True
3,和平公園東側,大安區,2024-06-17 19:21:21,和平東路二段118巷33號,11,121.54282,25.02351,7,4,True
4,辛亥復興路口西北側,大安區,2024-06-17 19:21:21,復興南路二段368號,16,121.54299,25.02153,7,9,True
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:13:15,臺大圖書館西南側,30,121.54031,25.01690,0,30,True
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,20,121.54347,25.01995,1,19,True
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,24,121.54242,25.01791,1,23,True
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:21:21,臺大體育館東側,40,121.53591,25.02112,8,32,True


In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   地區      1423 non-null   object 
 2   時間      1423 non-null   object 
 3   地點      1423 non-null   object 
 4   總車輛數    1423 non-null   int64  
 5   經度      1423 non-null   float64
 6   緯度      1423 non-null   float64
 7   可借      1423 non-null   int64  
 8   可還      1423 non-null   int64  
 9   狀態      1423 non-null   bool   
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [15]:
import numpy as np
less3_df = df1.query('可借<=3 or 可還<=3')  ### query裡面直接把整個敘述寫完整
less3_df.shape
less3_df


,站名,地區,時間,地點,總車輛數,經度,緯度,可借,可還,狀態
0,捷運科技大樓站,大安區,2024-06-17 19:21:21,復興南路二段235號前,28,121.54360,25.02605,1,27,True
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,16,121.54124,25.02429,2,14,True
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,11,121.54328,25.02429,10,1,True
6,復興南路二段340巷口,大安區,2024-06-17 19:17:36,復興南路二段342號,11,121.54326,25.02253,9,2,True
7,新生南路三段52號前,大安區,2024-06-17 19:17:30,新生南路三段52號,17,121.53407,25.02112,17,0,True
...,...,...,...,...,...,...,...,...,...,...
1415,臺大鄭江樓北側,臺大公館校區,2024-06-17 19:15:16,臺大新聞研究所南側,10,121.54305,25.01900,0,10,True
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:12:16,臺大電機二館南側,14,121.54247,25.01857,0,14,True
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:13:15,臺大圖書館西南側,30,121.54031,25.01690,0,30,True
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:21:21,臺大霖澤館南側,20,121.54347,25.01995,1,19,True


In [20]:
n=0
df3 = df1.query('地區=="大安區" and 可借==@n') ## 可以帶變數用@，字串要雙掛號(因為外面已經是單掛號)
df3

,站名,地區,時間,地點,總車輛數,經度,緯度,可借,可還,狀態
16,和平臥龍街口,大安區,2024-06-17 19:21:21,臥龍街1號,15,121.546705,25.024461,0,15,True
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,5,121.533100,25.020930,0,5,False
22,臺灣科技大學側門,大安區,2024-06-17 19:21:21,基隆路四段73巷與基隆路口,99,121.539730,25.012950,0,99,True
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:17:30,基隆路三段155巷57號西側,20,121.543820,25.014260,0,18,True
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,18,121.542910,25.015760,0,18,True
41,和平實小東北側,大安區,2024-06-17 19:21:21,敦南街73號,13,121.547040,25.021641,0,13,True
47,嘉興公園,大安區,2024-06-17 19:21:21,樂業街101巷3號西北側,44,121.552110,25.021000,0,39,True
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,14,121.548990,25.023700,0,14,True
68,敦化基隆路口,大安區,2024-06-17 19:17:30,敦化南路二段218號,38,121.548390,25.022150,0,36,True
69,安和路二段181巷口,大安區,2024-06-17 19:16:15,安和路二段185號,17,121.549880,25.027210,0,17,True


In [22]:
df3.shape

(27, 10)